# 3D Droplet Oscillation

Results published: hopefully at some point!

It is part of the BoSSS-long-term validation test suite, which consists of 
several computationally expensive test-cases (runtime in the order of days),
which are performed on a regular basis in order to validate the 
physical correctness of BoSSS simulations.


### Preliminaries

This example can be found in the source code repository as as `Droplet3D.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Timestepping;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D_FirstPeriodStudy");

Project name is set to 'OscillatingDroplet3D_FirstPeriodStudy'.
Opening existing database 'D:\local\OscillatingDroplet3D_FirstPeriodStudy'.


Overview on the available *Execution Queues* (aka. *Batch Processors*, aka. *Batch System*); these e.g. Linux HPC clusters on which compute jobs can be executed.

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,D:\local\binaries,False,LocalPC,dotnet,<null>,[ D:\local\ == ],,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\smuda\binaries,False,FDY-WindowsHPC,dotnet,,[ \\hpccluster\hpccluster-scratch\smuda\ == ],FDY\smuda,DC2,<null>,Normal,True


For this example (which is part of the BoSSS validation tests), a *default queue* is selected to run all jobs in the convergence study:

In [ ]:
var myBatch = ExecutionQueues[1];
//var myBatch = GetDefaultQueue();
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths
\\hpccluster\hpccluster-scratch\smuda\binaries,False,FDY-WindowsHPC,dotnet,FDY\smuda,DC2,<null>,Normal,True,[ \\hpccluster\hpccluster-scratch\smuda\ == ]


## Restart Infos

In [ ]:
OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D_FirstPeriodStudy");
//OpenOrCreateDatabase(@"\\dc1\userspace\smuda\Databases\OscillatingDroplet3D");

Opening existing database '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D_FirstPeriodStudy'.


In [ ]:
wmg.Sessions

#0: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta06_restart1*	05/20/2022 16:43:42	c9ef995c...
#1: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta03_restart1*	05/20/2022 16:43:34	72f1baaa...
#2: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta05	04/08/2022 11:32:02	9173b638...
#3: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta06	04/08/2022 11:31:47	6feb68ca...
#4: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta02	04/08/2022 11:32:41	012ffd75...
#5: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta03	04/08/2022 11:32:24	1de3f59e...
#6: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta07	04/08/2022 11:31:37	de146e9e...
#7: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta05*	04/08/2022 11:31:21	c6cd6b17...
#8: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta06*	04

In [ ]:
//wmg.Sessions.Pick(1).Delete(true);

Session 83c5ca35-e67b-4a49-967a-18c672b5052e deleted.


In [ ]:
var restartSessions = wmg.Sessions.Skip(8);
restartSessions

#0: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta03*	04/08/2022 11:30:37	56519a35...
#1: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta05*	04/08/2022 11:30:25	6f21cae2...
#2: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta06*	04/08/2022 11:30:08	71fcecb0...
#3: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta07*	04/08/2022 11:29:57	bb4b0796...


In [ ]:
List<(string sessionName, Guid restartID, string caseName)> cases = new List<(string sessionName, Guid restartID, string caseName)>();
int nCase = 0;
cases.Add((restartSessions.ElementAt(0).Name, restartSessions.ElementAt(0).ID, "m2_Oh01_eta03"));
cases.Add((restartSessions.ElementAt(1).Name, restartSessions.ElementAt(1).ID, "m2_Oh01_eta05"));
cases.Add((restartSessions.ElementAt(2).Name, restartSessions.ElementAt(2).ID, "m2_Oh01_eta06"));
cases.Add((restartSessions.ElementAt(3).Name, restartSessions.ElementAt(3).ID, "m2_Oh01_eta07"));

## Setup of control objects for all solver runs

In [ ]:
cases

index,Item1,Item2,Item3
0,OD3D_J686k3_wallBC_amr1_m2_Oh01_eta03,56519a35-fccc-4934-9bb1-f69e5a25a481,m2_Oh01_eta03
1,OD3D_J686k3_wallBC_amr1_m2_Oh01_eta05,6f21cae2-f82e-4059-a56c-139aee80115f,m2_Oh01_eta05
2,OD3D_J686k3_wallBC_amr1_m2_Oh01_eta06,71fcecb0-39df-4ce0-a65f-5bb7356a6be7,m2_Oh01_eta06
3,OD3D_J686k3_wallBC_amr1_m2_Oh01_eta07,bb4b0796-6373-4a28-8b9f-8168bc59b71c,m2_Oh01_eta07


In [ ]:
Dictionary<string, (double Ohnesorge, int AMRlevel, double dt, int timesteps)> casesSetUp = new Dictionary<string, (double Ohnesorge, int AMRlevel, double dt, int timesteps)>();

In [ ]:
casesSetUp.Add("m2_Oh01_eta07", (0.1, 1, 5e-3, 600));
casesSetUp.Add("m2_Oh01_eta06", (0.1, 1, 5e-3, 600));
casesSetUp.Add("m2_Oh01_eta05", (0.1, 1, 5e-3, 600));
casesSetUp.Add("m2_Oh01_eta03", (0.1, 1, 5e-3, 600));
casesSetUp.Add("m3_Oh01_eta07", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m3_Oh01_eta06", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m3_Oh01_eta05", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m3_Oh01_eta03", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m4_Oh01_eta07", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m4_Oh01_eta06", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m4_Oh01_eta05", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m4_Oh01_eta03", (0.1, 1, 5e-3, 300));
casesSetUp.Add("m4_Oh01_eta02", (0.1, 1, 5e-3, 300));

In [ ]:
List<XNSE_Control> Controls = new List<XNSE_Control>();
Controls.Clear();
int[] DegreeS = new int[] { 3 };
bool bNewton = false;

foreach(int k in DegreeS) {
int iCase = 0;
foreach(var myCase in cases) {

    string JobName = $"{myCase.sessionName}_restart1";

    Console.WriteLine($"Case {iCase+1}: " + JobName);
    iCase++;

    var C = new XNSE_Control();
    
    //C.SetGrid(grd);
    C.SetDGdegree(k);
    C.SessionName = JobName;
    
    //C.InitialValues.Add("Phi", casesPhi[myCase]);
    C.RestartInfo = new Tuple<Guid, BoSSS.Foundation.IO.TimestepNumber>(myCase.restartID, null);
    
    C.PhysicalParameters.IncludeConvection = true;
    C.PhysicalParameters.rho_A = 1;
    C.PhysicalParameters.rho_B = 0.001;
    C.PhysicalParameters.mu_A = casesSetUp[myCase.caseName].Ohnesorge;
    C.PhysicalParameters.mu_B = casesSetUp[myCase.caseName].Ohnesorge/1000;
    C.PhysicalParameters.reynolds_B = 0.0;
    C.PhysicalParameters.reynolds_A = 0.0;
    C.PhysicalParameters.Sigma = 1;
    C.PhysicalParameters.pFree = 0.0;
    C.PhysicalParameters.mu_I = 0.0;
    C.PhysicalParameters.lambda_I = 0.0;
    C.PhysicalParameters.lambdaI_tilde = -1.0;
    C.PhysicalParameters.betaS_A = 0.0;
    C.PhysicalParameters.betaS_B = 0.0;
    C.PhysicalParameters.betaL = 0.0;
    C.PhysicalParameters.theta_e = 1.5707963267948966;
    C.PhysicalParameters.sliplength = 0.0;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.useArtificialSurfaceForce = false;
    
    C.Option_LevelSetEvolution = BoSSS.Solution.LevelSetTools.LevelSetEvolution.StokesExtension;
    C.AdvancedDiscretizationOptions.SST_isotropicMode = SurfaceStressTensor_IsotropicMode.LaplaceBeltrami_ContactLine;
    C.LSContiProjectionMethod = ContinuityProjectionOption.ConstrainedDG;
    
    C.TimeSteppingScheme = TimeSteppingScheme.BDF3;
    if(bNewton) {
        C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    } else {
        C.NonLinearSolver.SolverCode = NonLinearSolverCode.Picard;
        C.NonLinearSolver.ConvergenceCriterion = 1e-9;
    }
    C.NonLinearSolver.MaxSolverIterations = 50;
    C.NonLinearSolver.MinSolverIterations = 3;
    C.Timestepper_BDFinit = TimeStepperInit.SingleInit;
    C.Timestepper_LevelSetHandling = LevelSetHandling.Coupled_Once;
    C.TimesteppingMode = AppControl._TimesteppingMode.Transient;
    C.dtFixed = casesSetUp[myCase.caseName].dt;
    C.NoOfTimesteps = casesSetUp[myCase.caseName].timesteps;
    
    int AMRlvl = casesSetUp[myCase.caseName].AMRlevel;
    if(AMRlvl > 0) {
        C.AdaptiveMeshRefinement = true;
        C.activeAMRlevelIndicators.Add(
            new AMRonNarrowband() { maxRefinementLevel = AMRlvl }
        );
    }
    
    C.PostprocessingModules.Add(new SphericalHarmonicsLogging() { MaxL = 8, RotSymmetric = true });
    C.PostprocessingModules.Add(new DropletMetricsLogging() { AxisSymmetric = true });
    C.PostprocessingModules.Add(new EnergyLogging());
    
    C.TracingNamespaces = "*";
    
    Controls.Add(C);
    
}
}

Case 1: OD3D_J686k3_wallBC_amr1_m2_Oh01_eta03_restart1
Case 2: OD3D_J686k3_wallBC_amr1_m2_Oh01_eta05_restart1
Case 3: OD3D_J686k3_wallBC_amr1_m2_Oh01_eta06_restart1
Case 4: OD3D_J686k3_wallBC_amr1_m2_Oh01_eta07_restart1


In [ ]:
int NC = Controls.Count;
for(int i = 0; i < NC; i++) {
    for(int j = 0; j < NC; j++) {
        if(i == j)
            Assert.IsTrue(Controls[i].Equals(Controls[j]), "Control is not self-equal for " + i);
        else
            Assert.IsFalse(Controls[i].Equals(Controls[j]), "Different Control are wrongly equal for " + i + " and " + j);
    }
}

## Launch Jobs

In [ ]:
Controls.Select(C => C.SessionName)

index,value
0,OD3D_J686k3_wallBC_amr1_m2_Oh01_eta03_restart1
1,OD3D_J686k3_wallBC_amr1_m2_Oh01_eta05_restart1
2,OD3D_J686k3_wallBC_amr1_m2_Oh01_eta06_restart1
3,OD3D_J686k3_wallBC_amr1_m2_Oh01_eta07_restart1


In [ ]:
foreach(var ctrl in Controls) {
    var oneJob              = ctrl.CreateJob();
    oneJob.NumberOfMPIProcs = 1;
    oneJob.Activate(myBatch); 
}

Deploying job OD3D_J686k3_wallBC_amr1_m2_Oh01_eta03_restart1 ... 
Set Database: { Session Count = 12; Grid Count = 103; Path = \\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D_FirstPeriodStudy }
Deploying executables and additional files ...
Deployment directory: \\hpccluster\hpccluster-scratch\smuda\binaries\OscillatingDroplet3D_FirstPeriodStudy-XNSE_Solver2022May20_164319
copied 50 files.
   written file: control.obj
deployment finished.

Deploying job OD3D_J686k3_wallBC_amr1_m2_Oh01_eta05_restart1 ... 
Set Database: { Session Count = 12; Grid Count = 103; Path = \\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D_FirstPeriodStudy }
Deploying executables and additional files ...
Deployment directory: \\hpccluster\hpccluster-scratch\smuda\binaries\OscillatingDroplet3D_FirstPeriodStudy-XNSE_Solver2022May20_164327
copied 50 files.
   written file: control.obj
deployment finished.

Deploying job OD3D_J686k3_wallBC_amr1_m2_Oh01_eta06_restart1 ... 
Set Database: { Session 

In [ ]:
//wmg.AllJobs

In [ ]:
// wait for all jobs to finish (up to 5 days, check every 30 minutes)
//BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600), PollingIntervallSeconds:(60*30));

In [ ]:
// detect failed Jobs in the job management
/*
var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
    .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
                  || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
suspects
*/

In [ ]:
//suspects.Count()

In [ ]:
//NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");

### Inspect the output of some arbitrary job:

In [ ]:
//BoSSSshell.WorkflowMgm.AllJobs.First().Value.ShowOutput();

In [ ]:
//wmg.Sessions[0].Export().WithSupersampling(2).Do()